In [2]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from glob import glob
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import json
import random
import pandas as pd
from tqdm import tqdm
import trimesh

import torch

from ANALYSIS.analysis_utils import (
    plot_panel_info,
    visualize_meshes_plotly,
    filter_segmentation_map,
    filter_segmentation_map_clusters,
    is_clockwise,
)

from env_constants import SEWFORMER_PROJ_ROOT, DATASET_ROOT, PYGARMENT_ROOT

sys.path.append(PYGARMENT_ROOT)

import pygarment as pyg

In [3]:
garment_df = pd.read_csv("garment_df_20250215_165422.csv")

filtered_combination_name_list = np.unique(garment_df[
    garment_df["mesh_filter_failed"] == False
]["matching_combination_name"]).tolist()

filtered_combination_path_list = list(map(
    lambda x : os.path.join(DATASET_ROOT, 'sewfactory', x),
    filtered_combination_name_list
))

In [4]:
import json
import numpy as np
import smplx
import torch
import trimesh

SMPLH_PATH = os.path.join(
    SEWFORMER_PROJ_ROOT, "Sewformer", "assets",
)

# Create SMPL-H model
model = smplx.create(
    model_path=SMPLH_PATH,
    model_type='smplh',  # Specifically use SMPL-H
    ext='pkl',
    gender='female',
    use_pca=False,  # Important: disable PCA for hand poses
    batch_size=1,
)

In [56]:
import numpy as np
import trimesh
import networkx as nx

def find_boundary_vertices(mesh):
    """
    Identifies boundary vertices in a manifold mesh.
    
    - Boundary edges are edges that belong to only one face.
    - Boundary vertices are the endpoints of boundary edges.
    
    Args:
        mesh (trimesh.Trimesh): The input mesh object.
    
    Returns:
        np.array: Indices of boundary vertices.
    """
    edges, counts = np.unique(np.sort(mesh.edges, axis=1), axis=0, return_counts=True)
    boundary_edges = edges[counts == 1]
    boundary_vertices = np.unique(boundary_edges)

    return boundary_vertices

def disassemble_mesh_to_panels(mesh, segmentation_list):
    """
    Disassembles a garment mesh into separate panel meshes and splits the segmentation list.

    Args:
        mesh (trimesh.Trimesh): The input garment mesh.
        segmentation_list (list): List classifying each vertex.

    Returns:
        tuple: (list of trimesh.Trimesh, list of segmentation lists for each panel)
    """
    # Step 1: Create a graph from the mesh connectivity
    mesh_graph = nx.Graph()

    # Add edges based on mesh faces (each face connects three vertices)
    for face in mesh.faces:
        mesh_graph.add_edge(face[0], face[1])
        mesh_graph.add_edge(face[1], face[2])
        mesh_graph.add_edge(face[2], face[0])

    # Step 2: Find connected components (each should be an individual panel)
    connected_components = list(nx.connected_components(mesh_graph))
    print(f"Found {len(connected_components)} separate panel meshes.")

    # Step 3: Extract panel sub-meshes
    panel_meshes = []
    panel_segmentation_list = []
    garment_to_panel_idx_map_list = []
    panel_to_garment_idx_map_list = []
    for component in connected_components:
        panel_vertex_indices = np.array(list(component))  # Original indices
        
        # Step 3.1: Create a mapping from original to new local indices
        # index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(panel_vertex_indices)}
        garment_to_panel_idx_map = {old_idx: new_idx for new_idx, old_idx in enumerate(panel_vertex_indices)}
        panel_to_garment_idx_map = {new_idx: old_idx for old_idx, new_idx in enumerate(panel_vertex_indices)}

        garment_to_panel_idx_map_list.append(garment_to_panel_idx_map)
        panel_to_garment_idx_map_list.append(panel_to_garment_idx_map)

        # Step 3.2: Extract the segmentation list for this panel
        this_panel_segmentation_list = [segmentation_list[idx] for idx in panel_vertex_indices]
        panel_segmentation_list.append(this_panel_segmentation_list)

        # Step 3.3: Get faces that only use the panel's vertices
        face_mask = np.isin(mesh.faces, panel_vertex_indices).all(axis=1)
        panel_faces = mesh.faces[face_mask]

        # Step 3.4: Reindex faces to local vertex indices
        reindexed_faces = np.vectorize(garment_to_panel_idx_map.get)(panel_faces)

        # Step 3.5: Create new mesh
        panel_mesh = trimesh.Trimesh(
            vertices=mesh.vertices[panel_vertex_indices],
            faces=reindexed_faces
        )
        panel_meshes.append(panel_mesh)

    return panel_meshes, panel_segmentation_list, garment_to_panel_idx_map_list, panel_to_garment_idx_map_list


In [83]:

IDX = 1551

IDX = random.randint(0, len(filtered_combination_path_list) - 1)
combination_path = filtered_combination_path_list[IDX]

print("IDX :", IDX)
print("combination_path :", combination_path)

spec_config_path = os.path.join(combination_path, "static", "spec_config.json")
with open(spec_config_path, "r") as f:
    spec_config = json.load(f)

garment_name_list = list(map(
    lambda x : os.path.basename(x["spec"].replace("\\", "/")),
    spec_config.values()
))

garment_dict = {}
for garment_name in garment_name_list :
    spec_file_path = os.path.join(
        combination_path, "static", f"{garment_name}_specification.json"
    )
    pattern = pyg.pattern.wrappers.VisPattern(spec_file_path)
    drawn_pattern_list = list(map(
        lambda pannel_name : pattern._draw_a_panel(
            pannel_name, apply_transform=False, fill=True
        ),
        pattern.panel_order()
    ))
    panel_svg_path_dict = {
        panel_name : pattern._draw_a_panel(
            panel_name, apply_transform=False, fill=True
        )
        for panel_name in pattern.panel_order()
    }

    stitch_dict = {
        i : v for i, v in enumerate(pattern.pattern['stitches'])
    }
    
    mesh = trimesh.load_mesh(os.path.join(
        combination_path, "static", f"{garment_name}_{garment_name}.obj",
    ), process=False)

    neighbor_idx_list = mesh.vertex_neighbors
    
    with open(
        os.path.join(
            combination_path, "static", f"{garment_name}_{garment_name}_segmentation.txt"
        ),
        "r"
    ) as f:
        mesh_segmentation_list = list(map(
            lambda x : x.strip(),
            f.readlines()
        ))
        vertex_mask_dict = {}
        for panel_name in panel_svg_path_dict.keys() :
            vertex_mask_dict[panel_name] = np.array(list(map(
                lambda x : x == panel_name,
                mesh_segmentation_list
            )))
        vertex_mask_dict["stitch"] = np.array(list(map(
            lambda x : x == "stitch",
            mesh_segmentation_list
        )))
        vertex_mask_dict["None"] = np.array(list(map(
            lambda x : x == "None",
            mesh_segmentation_list
        )))

    (
        panel_mesh_list, panel_segmentation_list,
        garment_to_panel_idx_map_list, panel_to_garment_idx_map_list
    ) = disassemble_mesh_to_panels(mesh, mesh_segmentation_list)
    
    panel_dict = {}
    for (
        panel_mesh, panel_segmentation_list, 
        garment_to_panel_idx_map, panel_to_garment_idx_map
    )in zip(
        panel_mesh_list, panel_segmentation_list,
        garment_to_panel_idx_map_list, panel_to_garment_idx_map_list
    ) :
        panel_name_candidate_list, count_list = np.unique(panel_segmentation_list, return_counts=True)
        panel_name = panel_name_candidate_list[np.argmax(count_list)]
        
        boundary_vert_idx_list = find_boundary_vertices(panel_mesh)
        
        
        overlapping_boundary_vert_count = np.sum(
            np.linalg.norm(
                panel_mesh.vertices[boundary_vert_idx_list[:-1]] - panel_mesh.vertices[boundary_vert_idx_list[1:]],
                axis=1
            ) < 1e-6
        )
        if overlapping_boundary_vert_count > 0 :
            print(panel_name, overlapping_boundary_vert_count)
                
        
        panel_dict[panel_name] = {
            "panel_mesh" : panel_mesh,
            "boundary_vert_idx_list" : boundary_vert_idx_list,
            "garment_to_panel_idx_map" : garment_to_panel_idx_map,
            "panel_to_garment_idx_map" : panel_to_garment_idx_map,
        }
        
    garment_dict[garment_name] = {
        "panel_svg_path_dict" : panel_svg_path_dict,
        "stitch_dict" : stitch_dict,
        "mesh" : mesh,
        "panel_dict" : panel_dict,
    }
    

IDX : 733
combination_path : /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_VAFQ1UNH68
Found 6 separate panel meshes.


In [81]:
panel_dict = list(
    list(garment_dict.values())[0]["panel_dict"].values()
)[0]

boundary_vertices = panel_dict["panel_mesh"].vertices[panel_dict["panel_boundary_vertices"]]



0

In [74]:
fig = visualize_meshes_plotly(
    panel_dict["panel_mesh"],
    vertices_list=[
        # panel_dict["panel_mesh"].vertices[panel_dict["panel_boundary_vertices"]]
        panel_dict["panel_mesh"].vertices[:3]
        
    ],
    vertices_color_list=["red"],
    show_edges=False,
    show=False
)
fig.show()

In [49]:

# IDX = 1551

# IDX = random.randint(0, len(filtered_combination_path_list) - 1)
# combination_path = filtered_combination_path_list[IDX]

# print("IDX :", IDX)
# print("combination_path :", combination_path)

# spec_config_path = os.path.join(combination_path, "static", "spec_config.json")
# with open(spec_config_path, "r") as f:
#     spec_config = json.load(f)

# garment_name_list = list(map(
#     lambda x : os.path.basename(x["spec"].replace("\\", "/")),
#     spec_config.values()
# ))



# with open(os.path.join(combination_path, "static", "static__body_info.json"), "r") as f:
#     static_body_data = json.load(f)
#     betas = torch.tensor(static_body_data['shape'], dtype=torch.float32).unsqueeze(0)
#     pose = torch.tensor(
#         np.deg2rad(static_body_data['pose']),
#         dtype=torch.float32
#     ).unsqueeze(0)  # Shape: (1, 52, 3)
#     transl = torch.tensor(static_body_data['trans'], dtype=torch.float32).unsqueeze(0)

#     body_pose = pose[0, 1:22].reshape(1, -1)  # Body joints (excluding global orientation)
#     left_hand_pose = pose[0, 22:37].reshape(1, -1)  # Left hand joints
#     right_hand_pose = pose[0, 37:52].reshape(1, -1)  # Right hand joints
#     global_orient = pose[0, 0].unsqueeze(0)  # Global orientation

#     # Get body mesh
#     output = model(
#         betas=betas,
#         body_pose=body_pose,
#         global_orient=global_orient,
#         left_hand_pose=left_hand_pose,
#         right_hand_pose=right_hand_pose,
#         # transl=transl
#     )
#     vertices = output.vertices.detach().numpy()[0]
#     faces = model.faces

#     # SCALE = 2 * transl.numpy()[0, 1] / (vertices[:, 1].max() - vertices[:, 1].min())
#     SCALE = 100
#     Z_OFFSET = -3
#     Y_OFFSET = 21

#     static_body_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
#     static_body_mesh.apply_scale(SCALE)

#     static_body_mesh.vertices += transl.numpy()
#     # static_body_mesh.vertices[:, 1] -= static_body_mesh.vertices[:, 1].min()
#     static_body_mesh.vertices[:, 2] += Z_OFFSET
#     static_body_mesh.vertices[:, 1] += Y_OFFSET

# mesh_dict = {}
# garment_dict = {}
# for garment_name in garment_name_list :
#     spec_file_path = os.path.join(
#         combination_path, "static", f"{garment_name}_specification.json"
#     )
#     pattern = pyg.pattern.wrappers.VisPattern(spec_file_path)
#     drawn_pattern_list = list(map(
#         lambda pannel_name : pattern._draw_a_panel(
#             pannel_name, apply_transform=False, fill=True
#         ),
#         pattern.panel_order()
#     ))
#     panel_svg_path_dict = {
#         panel_name : pattern._draw_a_panel(
#             panel_name, apply_transform=False, fill=True
#         )
#         for panel_name in pattern.panel_order()
#     }

#     stitch_dict = {
#         i : v for i, v in enumerate(pattern.pattern['stitches'])
#     }
    
#     mesh = trimesh.load_mesh(os.path.join(
#         combination_path, "static", f"{garment_name}_{garment_name}.obj",
#     ), process=False)

#     neighbor_idx_list = mesh.vertex_neighbors
    
#     with open(
#         os.path.join(
#             # combination_path, "static", f"{garment_name}_{garment_name}_segmentation_filtered.txt"
#             combination_path, "static", f"{garment_name}_{garment_name}_segmentation.txt"
#         ),
#         "r"
#     ) as f:
#         mesh_segmentation_list = list(map(
#             lambda x : x.strip(),
#             f.readlines()
#         ))
#         vertex_mask_dict = {}
#         for panel_name in panel_svg_path_dict.keys() :
#             vertex_mask_dict[panel_name] = np.array(list(map(
#                 lambda x : x == panel_name,
#                 mesh_segmentation_list
#             )))
#         vertex_mask_dict["stitch"] = np.array(list(map(
#             lambda x : x == "stitch",
#             mesh_segmentation_list
#         )))
#         vertex_mask_dict["None"] = np.array(list(map(
#             lambda x : x == "None",
#             mesh_segmentation_list
#         )))

#     fig = visualize_meshes_plotly(
#         [mesh],
#         show_edges=False,
#         # vertices_list = list(map(
#         #     lambda x : mesh.vertices[
#         #         x
#         #     ],
#         #     vertex_mask_dict.values()
#         # ))[:1],
#         # color_list = ["red", "blue", "green", "yellow", "white", "orange", "purple", "pink", "brown", "gray", "black", "white"],
        
#         vertices_list = [
#             mesh.vertices[vertex_mask_dict["stitch"]],
#             mesh.vertices[vertex_mask_dict["None"]],
#         ],
#         vertices_color_list = ["red", "blue"],
#         show=False
#     )
#     fig.show()

#     (
#         panel_mesh_list, panel_segmentation_list,
#         garment_to_panel_idx_map_list, panel_to_garment_idx_map_list
#     ) = disassemble_mesh_to_panels(mesh, mesh_segmentation_list)
    
#     panel_dict = {}
#     for (
#         panel_mesh, panel_segmentation_list, 
#         garment_to_panel_idx_map, panel_to_garment_idx_map
#     )in zip(
#         panel_mesh_list, panel_segmentation_list,
#         garment_to_panel_idx_map_list, panel_to_garment_idx_map_list
#     ) :
#         panel_vertex_mask_dict = {}
#         for panel_name in panel_svg_path_dict.keys() :
#             panel_vertex_mask_dict[panel_name] = np.array(list(map(
#                 lambda x : x == panel_name,
#                 panel_segmentation_list
#             )))
#         panel_vertex_mask_dict["stitch"] = np.array(list(map(
#             lambda x : x == "stitch",
#             panel_segmentation_list
#         )))
#         panel_vertex_mask_dict["None"] = np.array(list(map(
#             lambda x : x == "None",
#             panel_segmentation_list
#         )))
        
#         for panel_name, mask in panel_vertex_mask_dict.items() :
#             print(panel_name, mask.sum())
        
#         fig = visualize_meshes_plotly(
#             panel_mesh,
#             vertices_list = [
#                 panel_mesh.vertices[panel_vertex_mask_dict["stitch"]],
#                 panel_mesh.vertices[panel_vertex_mask_dict["None"]],
#             ],
#             vertices_color_list = ["red", "blue"],
#             show_edges=False,
#             show=False
#         )
#         fig.show()
        
        
#     garment_dict[garment_name] = {
#         "panel_svg_path_dict" : panel_svg_path_dict,
#         "stitch_dict" : stitch_dict,
#         "mesh" : mesh,
#         "panel_mesh_list" : panel_mesh_list,
#         "panel_segmentation_list" : panel_segmentation_list,
#     }
    

In [21]:
list(map(
    lambda x : mesh.vertices[
        x
    ],
    vertex_mask_dict.values()
))


[TrackedArray([[ 71.534477,  79.894379, -30.163786],
               [ 72.194023,  80.347786, -32.65115 ],
               [ 71.373062,  80.751518, -31.604897],
               ...,
               [ 67.776794,  21.107845, -33.875992],
               [ 67.288666,  20.607347, -34.90855 ],
               [ 66.696274,  20.134119, -35.898266]]),
 TrackedArray([[ 60.822735,  75.276833, -15.802821],
               [ 60.283657,  76.008743, -15.494904],
               [ 59.928192,  74.232895, -15.20603 ],
               ...,
               [ 67.23082 ,  60.707123, -34.6385  ],
               [ 66.479416,  40.416161, -35.763733],
               [ 42.37682 ,  53.735226, -43.135204]]),
 TrackedArray([[ 32.54216 ,  59.391609, -34.140446],
               [ 32.799213,  60.520199, -34.097122],
               [ 31.902117,  60.11599 , -34.00824 ],
               ...,
               [ 66.462044,  69.332375, -18.375847],
               [ 67.135773,  72.891144, -18.616676],
               [ 60.794945,  72.543

In [27]:
vertex_mask_dict.keys()

dict_keys(['skirt_back', 'skirt_front', 'top_back', 'top_front'])